In [1]:
! pip install -q kaggle tensorflow numpy pandas

from google.colab import files
files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets download -d deadskull7/fer2013
! unzip -o fer2013.zip
! rm fer2013.zip

print("\n--- Download Complete ---")

Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
Dataset URL: https://www.kaggle.com/datasets/deadskull7/fer2013
License(s): CC0-1.0
  0% 0.00/96.6M [00:00<?, ?B/s]
100% 96.6M/96.6M [00:00<00:00, 1.63GB/s]
Archive:  fer2013.zip
  inflating: fer2013.csv             

--- Download Complete ---


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import VGG16
import os

DATASET_PATH = 'fer2013.csv'
IMG_SIZE = 48
NUM_CLASSES = 7
BATCH_SIZE = 128
EPOCHS = 50
MODEL_NAME = 'facial_emotion_vgg16_7class.h5'

print(f"Loading ALL 7-Class data from: {DATASET_PATH}...")
data = pd.read_csv(DATASET_PATH)

pixels = data['pixels'].tolist()
emotions = data['emotion'].values

X = []
for pixel_sequence in pixels:
    face = np.fromstring(pixel_sequence, dtype=int, sep=' ').reshape(IMG_SIZE, IMG_SIZE)
    face_rgb = np.stack((face,)*3, axis=-1)
    X.append(face_rgb)

X = np.array(X).astype('float32') / 255.0
y = to_categorical(emotions, num_classes=NUM_CLASSES)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=emotions)

print(f"Training Data Shape (RGB 3-channel): {X_train.shape}")


def create_vgg16_transfer_model(input_shape, num_classes):
    conv_base = VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=input_shape)

    conv_base.trainable = False

    model = Sequential([
        conv_base,
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

input_shape = (IMG_SIZE, IMG_SIZE, 3)
model = create_vgg16_transfer_model(input_shape, NUM_CLASSES)

datagen = ImageDataGenerator(
    rotation_range=15, width_shift_range=0.1, height_shift_range=0.1,
    shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest'
)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

print("\n--- Starting 7-Class Transfer Learning Training ---")
model.fit(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

model.save(MODEL_NAME)
print(f"\nFINAL MODEL SAVED: {MODEL_NAME}")

Loading ALL 7-Class data from: fer2013.csv...
Training Data Shape (RGB 3-channel): (28709, 48, 48, 3)

--- Starting 7-Class Transfer Learning Training ---


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 44s 182ms/step - accuracy: 0.2745 - loss: 1.7907 - val_accuracy: 0.3578 - val_loss: 1.6177
Epoch 2/50
  1/224 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.3125 - loss: 1.6982

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


224/224 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.3125 - loss: 1.6982 - val_accuracy: 0.3559 - val_loss: 1.6190
Epoch 3/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 36s 162ms/step - accuracy: 0.3465 - loss: 1.6483 - val_accuracy: 0.3693 - val_loss: 1.5897
Epoch 4/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.3672 - loss: 1.6102 - val_accuracy: 0.3679 - val_loss: 1.5935
Epoch 5/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 33s 148ms/step - accuracy: 0.3644 - loss: 1.6190 - val_accuracy: 0.3865 - val_loss: 1.5769
Epoch 6/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.2734 - loss: 1.7965 - val_accuracy: 0.3824 - val_loss: 1.5789
Epoch 7/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 34s 152ms/step - accuracy: 0.3723 - loss: 1.6019 - val_accuracy: 0.3990 - val_loss: 1.5401
Epoch 8/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.4062 - loss: 1.6463 - val_accuracy: 0.3990 - val_loss: 1.5399
Epoch 9/50
224/224 ━━━━━━━━━━━━━━━━━━━━ 34s 152ms/step - accuracy: 0.3734 - loss: 1.5957 - val_accurac


FINAL MODEL SAVED: facial_emotion_vgg16_7class.h5
